# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Upload kaggle.json file
from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle
!pip install -q pydub
!pip install -q transformers
!pip install -q tensorboard

!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification
!unzip -q gtzan-dataset-music-genre-classification.zip
!mv Data data
!mv data/genres_original data/audio

# jazz.00054.wav is corrupted so we need to delete it
!rm ./data/audio/jazz/jazz.00054.wav

In [ ]:
# Set this to drive location of colab_version
%cd /content/drive/MyDrive/DL/colab_version

from utils import set_data_root_dir
set_data_root_dir('/content')

# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [126]:
import torch

from utils.train_utils import train
from utils.data_utils import setup_dataset, read_into_df, get_train_val, RawDS
from models.CombinedClassifier import CombinedClassifier, ASTPipeline

In [127]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device: %s" % device)

Using device: cpu


In [128]:
setup_dataset()
df = read_into_df()
df.head()

Dataset already setup!


# Combined Classifier

In [ ]:
dataset = RawDS(df)
train_dl, val_dl = get_train_val(dataset, train_split=0.8, batch_size=8)

model = CombinedClassifier(device)
model.to(device)

num_epochs = 20
lr = 0.001
train(model, train_dl, val_dl, num_epochs, lr, device)